# Exploring Terms in the Encyclopaedia Britannica

## Similar terms within an edition - Gensim - Doc2Vec

In this notebook we are going to find similar articles/terms with the dataframe that we have obtained either with the posprocess_eb.py script or Merging_EB_Terms.ipynb notebooks. Both methods obtain the same dataframe. 

We have selected the first Edition for this explorations, but we can run this notebook with any of the other editions.

**Remark**: Edition 1, has 3 volumes, and it was printed twice, in 1771 and 1773. 

These are the explorations that we are going to do:
- Create a new dataframe selecting just the first 100 elements of the first vol of 1771. This is the df that we are going to use for the rest of this notebook.
- Create a training corpus with the previous dataframe
- Create a doc2vec model with the training corpus
- Saving the model to disk
- Testing the model - using the term ALCOHOL. 



### Loading the necessary libraries

In [1]:
import yaml
import matplotlib.pyplot as plt
import numpy as np
import collections
import matplotlib as mpl

In [2]:
import networkx as nx
import matplotlib.pyplot as plt

In [3]:
import pandas as pd
from yaml import safe_load
from pandas.io.json import json_normalize

In [4]:
import gensim
from gensim.models.doc2vec import Doc2Vec

In [5]:
from doc2vec_prep import stem_text, clean_text, generate_documents_df

/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/anaconda3/envs/mypython3/lib/python3.7/site-packages/tensorflow/python/f

In [6]:
from tqdm import tqdm
import os


#### Hyperparameters

In [7]:
# Init the Doc2Vec model
hyperparams  = {
    'dm': 1,
    'vector_size': 300,
    'window': 5,
    'alpha': 0.025,
    'min_alpha': 0.00025,
    'min_count': 2,
    'workers': 8
}

### Functions

In [8]:
def get_document(df, index):
    print("INDEX IS %s" %index)
    term = df.loc[index]["term"]
    definition = df.loc[index]["definition"]
    return term, definition

In [9]:
def most_similar(model, text, clean_func=clean_text, topn=None):
    vector = model.infer_vector(clean_func(text), epochs=100, alpha=model.alpha, min_alpha=model.min_alpha)
    simdocs = model.docvecs.most_similar(positive=[vector], topn=topn)
    return simdocs

In [10]:
def load_model(filename):
    try:
        return Doc2Vec.load(os.path.join(MODEL_PATH, filename), mmap='r')
    except:
        return None


## We have dataframe with these columns

- definition:           Definition of a term
- editionNum:           1,2,3,4,5,6,7,8
- editionTitle:         Title of the edition
- header:               Header of the page's term                                  
- place:                Place where the volume was edited (e.g. Edinburgh)                                    
- relatedTerms:         Related terms (see X article)  
- altoXML:              File Path of the XML file from which the term belongs       
- term:                 Term name                            
- positionPage:         Position of ther term in the page     
- startsAt:             Number page in which the term definition starts 
- endsAt:               Number page in which the term definition ends 
- volumeTitle:          Title of the Volume
- typeTerm:             Type of term [Topic| Articles]                                       
- year:                 Year of the edition
- volumeNum:            Volume number (e.g. 1)
- letters:              leters of the volume (A-B)
- part:                 Part of the volume (e.g 1)
- supplement:           Supplement's Title
- supplementsTo:        It suppelements to editions [1, 2, 3....]
- numberOfWords:        Number of words per term definition
- numberOfTerms:        Number of terms per page
- numberOfPages:        Number of pages per volume

### 1. Load dataframe from JSON file

In [11]:
df = pd.read_json('../../results_NLS/results_eb_1_edition_dataframe', orient="index") 

Now we are going to oder the columns of our dataframe and visualise it. 

In [12]:
df = df[["term", "definition", "relatedTerms", "header", "startsAt", "endsAt", "numberOfTerms","numberOfWords", "numberOfPages", \
             "positionPage", "typeTerm", "editionTitle", "editionNum", "supplementTitle", "supplementsTo",\
             "year", "place", "volumeTitle", "volumeNum", "letters", "part", "altoXML"]].reset_index(drop=True)

df


,term,definition,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,numberOfPages,positionPage,...,editionNum,supplementTitle,supplementsTo,year,place,volumeTitle,volumeNum,letters,part,altoXML
0,OR,"A NEW A D I C T I A A, the name of several riv...",[],EncyclopaediaBritannica,15,15,22,54,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
1,AABAM,"a term, among alchemifts, for lead,",[],EncyclopaediaBritannica,15,15,22,6,832,1,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
2,AACH,the name of a town and river in Swabia. It is ...,[],EncyclopaediaBritannica,15,15,22,17,832,2,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
3,AADE,"the name of two rivers, one in the country of ...",[],EncyclopaediaBritannica,15,15,22,19,832,3,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
4,AAHUS,a small town and diftrift in Weftphalia.,[],EncyclopaediaBritannica,15,15,22,7,832,4,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27159,ZUYDERSEE,"a great bay of the German ocean, which lies in...",[],ZoDZYG,857,857,27,66,864,22,...,1,,[],1773,London,"Encyclopaedia Britannica: or, A dictionary of ...",3,M-Z,0,144850368/alto/188375020.34.xml
27160,ZWEIBRUGGEN,"a county of the palatinate of the Rhine, in Ge...",[SQALVS],ZoDZYG,857,857,27,23,864,23,...,1,,[],1773,London,"Encyclopaedia Britannica: or, A dictionary of ...",3,M-Z,0,144850368/alto/188375020.34.xml
27161,ZYGOMA,in anatomy. See Anatomy p. 152.,[],ZoDZYG,857,857,27,6,864,24,...,1,,[],1773,London,"Encyclopaedia Britannica: or, A dictionary of ...",3,M-Z,0,144850368/alto/188375020.34.xml
27162,ZYGOMATICUS,"in anatomy,. See Anatomy, p. 306,",[ANATOMY],ZoDZYG,857,857,27,6,864,25,...,1,,[],1773,London,"Encyclopaedia Britannica: or, A dictionary of ...",3,M-Z,0,144850368/alto/188375020.34.xml


### 2.  Selecting just the 100 first elements of  the first volume of 1771

In [13]:
df_1771_small = df[(df['year'] == 1771)]
df_1771_small = df_1771_small.head(100).reset_index(drop=True)


In [14]:
df_1771_small

,term,definition,relatedTerms,header,startsAt,endsAt,numberOfTerms,numberOfWords,numberOfPages,positionPage,...,editionNum,supplementTitle,supplementsTo,year,place,volumeTitle,volumeNum,letters,part,altoXML
0,OR,"A NEW A D I C T I A A, the name of several riv...",[],EncyclopaediaBritannica,15,15,22,54,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
1,AABAM,"a term, among alchemifts, for lead,",[],EncyclopaediaBritannica,15,15,22,6,832,1,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
2,AACH,the name of a town and river in Swabia. It is ...,[],EncyclopaediaBritannica,15,15,22,17,832,2,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
3,AADE,"the name of two rivers, one in the country of ...",[],EncyclopaediaBritannica,15,15,22,19,832,3,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
4,AAHUS,a small town and diftrift in Weftphalia.,[],EncyclopaediaBritannica,15,15,22,7,832,4,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082904.34.xml
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,ABEYANCE,"in law, the expedtancy of an edate. Thus if la...",[],ABE,18,18,16,36,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082943.34.xml
96,ABHEL,"in botany, an obsolete name of the fabina or s...",[],ABE,18,18,16,14,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082943.34.xml
97,ABIES,"the sir-tree, in botany, belongs to the monaec...",[],ABE,18,18,16,16,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082943.34.xml
98,PINUS,"of which it is a (pecies,",[],ABE,18,18,16,6,832,0,...,1,,[],1771,Edinburgh,"Encyclopaedia Britannica; or, A dictionary of ...",1,A-B,0,144133901/alto/188082943.34.xml


### 2.1 Counting the number of terms

**Remember**: A term can appear in more than once  per eddition. 

In [15]:
len(df_1771_small)

100

### 3. Creating Document embeddings


Document Embeddings (doc2vec), an unsupervised method for learning vector representations for variable-length pieces of texts such as sentences and document. We have used the gensim Python Library  to create a document embedding model, using the term and definition information of the **df_1771_small**, which only has 100 elements. 



#### 3.1 Train Corpus

First we have to create a training corpus (train_documents) with the elements of the **df_1771_small**. We are going to use only the information of **term** and **definition** text, to create our "text" per row of this dataframe. Furthermore, we are going to clean this dataset, by applying a serie of transformations (removing stop words, normalise, tokenize,  etc.). We are going to select the text that has a minimum length of 5 words. 

In [16]:
train_documents = list(tqdm(generate_documents_df(df_1771_small, clean_text, min_words=5)))
#train_documents = list(tqdm(generate_documents_df(df_1771_small, stem_text, min_words=5)))

1it [00:00,  7.53it/s]

Preprocessing function: clean_text
Minimum document length: 5 words


83it [00:02, 29.66it/s]

Generated 83 description terms


#### 3.2 Creating the model

Once we have created our trained corpus, we are going to create our model. In this step we are going to create a document embedding per element in our training corpus. 

In [17]:
print(f'Created {len(train_documents)} tagged documents.')
model = Doc2Vec(**hyperparams)
print('Build vocabulary')
model.build_vocab(train_documents)
for epoch in range(100):
    print(f'Train model: epoch={epoch}')
    model.train(train_documents, total_examples=model.corpus_count, epochs=1)
    model.alpha -= 0.0002
    model.min_alpha = model.alpha

Created 83 tagged documents.
Build vocabulary
Train model: epoch=0
Train model: epoch=1
Train model: epoch=2
Train model: epoch=3
Train model: epoch=4
Train model: epoch=5
Train model: epoch=6
Train model: epoch=7
Train model: epoch=8
Train model: epoch=9
Train model: epoch=10
Train model: epoch=11
Train model: epoch=12
Train model: epoch=13
Train model: epoch=14
Train model: epoch=15
Train model: epoch=16
Train model: epoch=17
Train model: epoch=18
Train model: epoch=19
Train model: epoch=20
Train model: epoch=21
Train model: epoch=22
Train model: epoch=23
Train model: epoch=24
Train model: epoch=25
Train model: epoch=26
Train model: epoch=27
Train model: epoch=28
Train model: epoch=29
Train model: epoch=30
Train model: epoch=31
Train model: epoch=32
Train model: epoch=33
Train model: epoch=34
Train model: epoch=35
Train model: epoch=36
Train model: epoch=37
Train model: epoch=38
Train model: epoch=39
Train model: epoch=40
Train model: epoch=41
Train model: epoch=42
Train model: epoch

#### 3.3 Saving the model to disk

In [18]:
# Save the model
model_path = os.path.join("../../results_NLS/", 'doc2vec_df_1771_small.model')
model.save(model_path)
print(f'Saved model to {model_path}')

Saved model to ../../results_NLS/doc2vec_df_1771_small.model


#### 3.5 Obtaining a list of elements in 

In [19]:
list_of_terms=df_1771_small["term"].to_list()
list_of_terms

['OR',
 'AABAM',
 'AACH',
 'AADE',
 'AAHUS',
 'AAM',
 'AAR',
 'AARSEO',
 'AATTER',
 'AB',
 'ASTRONOMY',
 'AB',
 'ABACATUAIA',
 'ABAC',
 'ABACH',
 'ABACISCUS',
 'ABACO',
 'ABACOT',
 'ABACTORES',
 'ABACTUS',
 'ABACUS',
 'ABACUS',
 'ABACUS',
 'ABACUS',
 'ABADAN',
 'ABADDON',
 'ABADIR',
 'ABACRE',
 'ABAFT',
 'ABAI',
 'ABAISSE',
 'ABALIENATION',
 'ABANBO',
 'ABANCAI',
 'ABANO',
 'ABAPTISTON',
 'ABARCA',
 'ABARTICULATION',
 'ABAS',
 'ABASCIA',
 'ABAISED',
 'ABASING',
 'ABASSI',
 'ABATAMENTUM',
 'ABATE',
 'ABATE',
 'ABATEMENT',
 'ABATEMENT',
 'ABATIS',
 'AB',
 'ABAVO',
 'ABAYANCE',
 'ABB',
 'ABBA',
 'ABBAT',
 'ABBATIS',
 'ABBEFORD',
 'ABBESS',
 'ABBEVILLE',
 'ABBEY',
 'ABBEYBOYLE',
 'ABBOT',
 'ABBREVIATE',
 'ABBREVIATOR',
 'ABBREVIATORS',
 'ABBREVOIR',
 'ABBROCHMENT',
 'ABBUTTALS',
 'ABCASSES',
 'ABCDARIA',
 'ABCDARY',
 'ABDALS',
 'ABDELAVI',
 'ABDEST',
 'ABDICARIAN',
 'ABDICATION',
 'ABDOMEN',
 'ABDUCTION',
 'ABDUCTION',
 'ABDUCTOR',
 'ABELTREE',
 'ABELMOSCH',
 'ABENSBURG',
 'ABERBROTHOCK',


#### 3.4 Testing the model - Similar Terms

Selecting the defintion of term alchohol, which is position 7 inside **df_1771_small**. 

In [20]:
term=df_1771_small.loc[22]["term"]
term

'ABACUS'

In [21]:
text=df_1771_small.loc[22]["definition"]
text

'is also the name of an ancient instrument for facilitating operations in arithmetic. It is vadoully contrived. That chiefly used in Europe is made by drawing any number of parallel lines at the di(lance of two diameters of one of the counters used in the calculation. A counter placed on.the lowed line, signifies r; on the sd, 10; on the 3d, 100; on the 4th, 1000, &c. In the intermediate spaces, the same counters are eflimated at one Jialf of the value of the line immediately superior, viz. between the id and 2d, 5; between the 2d and 3d, 50, &c. See plate I. fig. 2. A B, where the same number, 1768 for example, is represented under both by different dispositions of the counters.'

In [22]:
#model=load_model('../../results_NLS/doc2vec_df_1771_small.model')
cleaned_text = clean_text(text)
#cleaned_tex = stem_text(text)
# Just going to take the firs 10 -- so topn=10
simdocs=most_similar(model, text, topn=10)

In [23]:
print("#### TEST 1 -- Doc2Vec -- Printing the details of the 10 most similar documents using Doc2Vec ")
for doc_id , rank in simdocs:
    term, definition = get_document(df_1771_small, doc_id)
    print("!! Using DocVec --- Document_id: %s - Rank %s - Details: Term %s, Definition: %s" %(doc_id, rank, term, definition))
    print("---")

#### TEST 1 -- Doc2Vec -- Printing the details of the 10 most similar documents using Doc2Vec 
INDEX IS 22
!! Using DocVec --- Document_id: 22 - Rank 0.9947737455368042 - Details: Term ABACUS, Definition: is also the name of an ancient instrument for facilitating operations in arithmetic. It is vadoully contrived. That chiefly used in Europe is made by drawing any number of parallel lines at the di(lance of two diameters of one of the counters used in the calculation. A counter placed on.the lowed line, signifies r; on the sd, 10; on the 3d, 100; on the 4th, 1000, &c. In the intermediate spaces, the same counters are eflimated at one Jialf of the value of the line immediately superior, viz. between the id and 2d, 5; between the 2d and 3d, 50, &c. See plate I. fig. 2. A B, where the same number, 1768 for example, is represented under both by different dispositions of the counters.
---
INDEX IS 23
!! Using DocVec --- Document_id: 23 - Rank 0.9902730584144592 - Details: Term ABACUS, Defin